### Install the required Python modules/SDKs 

In [3]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

## Import libraries

In [18]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os, time, uuid, glob

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


## Creating Training & Prediting Client with credentials

In [5]:
TRAINING_ENDPOINT = "https://longthcustomvision.cognitiveservices.azure.com/"
training_key = "24042cf430334292b60e0edd3809fa43"
training_resource_id = '/subscriptions/21c53bc7-9f96-4753-9901-99cd641ad4e7/resourceGroups/ODL-AIND-197885/providers/Microsoft.CognitiveServices/accounts/longthcustomvision'

PREDICTION_ENDPOINT = 'https://longthcustomvision-prediction.cognitiveservices.azure.com/'
prediction_key = "cf098aaaaf7043ff92ebf339cc5cbb52"
prediction_resource_id = "/subscriptions/21c53bc7-9f96-4753-9901-99cd641ad4e7/resourceGroups/ODL-AIND-197885/providers/Microsoft.CognitiveServices/accounts/longthcustomvision-Prediction"

In [7]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)
trainer.api_version

'3.4-preview'

In [8]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)
predictor.api_version

'3.1'

## Get project ID and iteration ID from portal

In [13]:
project_id = trainer.get_projects()[0].id # Assume this is the only project on portal
print("Project ID: ", project_id)

Project ID:  974e8f6b-e5bb-432f-ac28-cb20ef4273a6


In [16]:
iteration_list = trainer.get_iterations(project_id)
chosen_iteration_id = None
for iteration in iteration_list:
    if iteration.name == "Iteration 1":
        chosen_iteration_id = iteration.id

## Publish api with selected training iteration

In [17]:
publish_iteration_name = "lighter-detection-75"
trainer.publish_iteration(project_id, chosen_iteration_id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Perform prediction on test baggage images

In [30]:
def perform_prediction(predictor, publish_iteration_name, image_folder="./", threshold=0.8):
    list_images = glob.glob(os.path.join(image_folder, '*.jpg'))
    for image_file_path in list_images:
        print("Predicting image: ", image_file_path)
        with open(image_file_path, "rb") as image_contents:
            results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
            # Display the results.
            for prediction in results.predictions:
                if prediction.probability >= threshold:
                    print("\t" + prediction.tag_name +
                          ": {0:.2f}%".format(prediction.probability * 100))

In [31]:
perform_prediction(predictor, publish_iteration_name)

Predicting image:  ./lighter_test_set_4of5.jpg
	lighter: 99.41%
	lighter: 88.50%
Predicting image:  ./lighter_test_set_5of5.jpg
	lighter: 86.63%
Predicting image:  ./lighter_test_set_2of5.jpg
	lighter: 98.83%
	lighter: 96.99%
	lighter: 93.57%
	lighter: 92.49%
Predicting image:  ./lighter_test_set_3of5.jpg
	lighter: 98.38%
	lighter: 91.21%
Predicting image:  ./lighter_test_set_1of5.jpg
	lighter: 99.79%
	lighter: 95.34%
	lighter: 91.43%
